<a href="https://colab.research.google.com/github/choiyoumin/UWC-2025-AI-FireWatcher/blob/main/UWC_2025_AI_%26_%EB%AC%B4%EC%9D%B8%EC%9D%B4%EB%8F%99%EC%B2%B4_%ED%93%A8%EC%B2%98_%ED%95%B4%EC%BB%A4%ED%86%A4_%EC%86%8C%EC%8A%A4_%EC%BD%94%EB%93%9C_FireWatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [요청] LSH0619. Python을 이용한 2025년 AI&무인이동체 퓨처 해커톤 (AI 드론 산불 탐지 및 경고 알림)

## 개발환경 설정

In [3]:
import os
from google.colab import drive
import glob
from IPython.display import HTML
from base64 import b64encode
import os
import json
import sys
import requests

# 구글 드라이브 연결
drive.mount('/content/drive')

# 주요 설정
serviceName = 'FireWatcher'
contextPath = f"/content/drive/MyDrive/COLAB/testSys-{serviceName}"

globalVar = {
    'ctxPath':  f"{contextPath}"
    , 'inpPath': f"{contextPath}/INPUT"
    , 'outPath': f"{contextPath}/OUTPUT"
    , 'figPath': f"{contextPath}/FIG"
    , 'srcPath': f"{contextPath}/droneSmokeFireDetection"
}

for key, val in globalVar.items():
    if key.__contains__('Path'):
        os.makedirs(val, exist_ok=True)
        print(f"[CHECK] {key} : {val}")

# 작업 경로 설정
os.chdir(f"{globalVar['ctxPath']}")
print(f"[CHECK] getcwd : {os.getcwd()}")

# 옵션 설정
sysOpt = {
    'telegram': [
        {'token': '7068921384:AAFjlx_qdejg4V7HVaNnnLq-5cegX5efc5s', 'chat_id': '-1002840201938'}, # 연기/산불 탐지 그룹
    ],
    'isInit': False,
}

# 구글 드라이브 삭제
# ! rm -rf '/content/drive/MyDrive/COLAB/testSys-LSH0619'

# 구글 코랩 세션 무한 연결
# function PreventDisconnection(){
#     document.querySelector("colab-toolbar-button#connect").click()
#     console.log("클릭이 완료되었습니다.");
# }
# setInterval(PreventDisconnection, 60 * 10000)

Mounted at /content/drive
[CHECK] ctxPath : /content/drive/MyDrive/COLAB/testSys-FireWatcher
[CHECK] inpPath : /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT
[CHECK] outPath : /content/drive/MyDrive/COLAB/testSys-FireWatcher/OUTPUT
[CHECK] figPath : /content/drive/MyDrive/COLAB/testSys-FireWatcher/FIG
[CHECK] srcPath : /content/drive/MyDrive/COLAB/testSys-FireWatcher/droneSmokeFireDetection
[CHECK] getcwd : /content/drive/MyDrive/COLAB/testSys-FireWatcher


## 개발환경 구축

In [4]:
# 소스코드 다운로드
! gdown https://drive.google.com/uc?id=1tRQdMF6knszG1P1rAeR7r6ZLkHdhlDKL
! unzip droneSmokeFireDetection.zip -d droneSmokeFireDetection

# 라이브러리 다운로드/설치
! pip install ultralytics
! pip install python-telegram-bot
! pip install pytz

# 샘플파일 다운로드
os.chdir(f"{globalVar['inpPath']}")
print(f"[CHECK] getcwd : {os.getcwd()}")
! gdown https://drive.google.com/uc?id=18omItBxmzOzRBssdbd7EgCkJTiM5mkYz
# ! gdown https://drive.google.com/uc?id=1k3NJkyaVg8M7ffzhPSo_wduGFs7xCNFe
# ! gdown https://drive.google.com/uc?id=1M6MxbCLeD9wbrk39cFdASHhRAhK8rgsC
# ! gdown https://drive.google.com/uc?id=1tc4IN4J4VRqPHwBu_6Je7LAtOKLNJsDn

Downloading...
From (original): https://drive.google.com/uc?id=1tRQdMF6knszG1P1rAeR7r6ZLkHdhlDKL
From (redirected): https://drive.google.com/uc?id=1tRQdMF6knszG1P1rAeR7r6ZLkHdhlDKL&confirm=t&uuid=318e52bb-2cfd-44b8-ae96-f20a95034574
To: /content/drive/MyDrive/COLAB/testSys-FireWatcher/droneSmokeFireDetection.zip
100% 109M/109M [00:02<00:00, 54.4MB/s] 
Archive:  droneSmokeFireDetection.zip
   creating: droneSmokeFireDetection/datasets/
   creating: droneSmokeFireDetection/datasets/fire-8/
  inflating: droneSmokeFireDetection/datasets/fire-8/data.yaml  
  inflating: droneSmokeFireDetection/datasets/fire-8/README.dataset.txt  
  inflating: droneSmokeFireDetection/datasets/fire-8/README.roboflow.txt  
   creating: droneSmokeFireDetection/datasets/fire-8/test/
   creating: droneSmokeFireDetection/datasets/fire-8/test/images/
  inflating: droneSmokeFireDetection/datasets/fire-8/test/images/fire1_mp4-26_jpg.rf.5a09c11c9facf23a9413ca63bc2a6085.jpg  
  inflating: droneSmokeFireDetection/dataset

## 비즈니스 로직

In [5]:
# 주요 라이브러리
import ultralytics
from ultralytics import YOLO
from datetime import datetime
import asyncio
import telegram
import re
import pytz
ultralytics.checks()

# 함수 선언
async def sendTelegram(cfgList, msg):
    for cfgInfo in cfgList:
        try:
            bot = telegram.Bot(token=cfgInfo['token'])
            await bot.send_message(chat_id=cfgInfo['chat_id'], text=msg)
        except Exception as e:
            print(f"Exception : {e}")
            # pass

Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.0/107.7 GB disk)


In [6]:
os.chdir(f"{globalVar['ctxPath']}/droneSmokeFireDetection")
print(f"[CHECK] getcwd : {os.getcwd()}")

# Python SDK 방법
modeInfo = f"{globalVar['ctxPath']}/droneSmokeFireDetection/runs/detect/train/weights/best.pt"

# 드론 산불
sourceInfo = f"{globalVar['inpPath']}/20250707_drone_fire.mp4"

outInfo = f"{globalVar['outPath']}/{os.path.splitext(os.path.basename(sourceInfo))[0]}"

model = YOLO(modeInfo)
# results = model.predict(source=sourceInfo, conf=0.25, stream=True, save=True, project=os.path.dirname(outInfo), name=os.path.basename(outInfo))
results = model.predict(source=sourceInfo, conf=0.10, stream=True, save=True, project=os.path.dirname(outInfo), name=os.path.basename(outInfo))

cnt = 0
for r in results:
    for box in r.boxes:
        clsId = int(box.cls[0])
        clsName = model.names[clsId].lower()
        clsConf = int(box.conf[0].item() * 100)
        if clsName not in ['fire', 'smoke']: continue

        if re.search('smoke', clsName, re.IGNORECASE):
            msg = f"[{datetime.now(pytz.utc).astimezone(pytz.timezone('Asia/Seoul')).strftime('%Y-%m-%d %H:%M:%S')}] ⚠️ 연기 {clsName} 탐지 (발생확률 {clsConf}%)"
        elif re.search('fire', clsName, re.IGNORECASE):
            msg = f"[{datetime.now(pytz.utc).astimezone(pytz.timezone('Asia/Seoul')).strftime('%Y-%m-%d %H:%M:%S')}] 🔥 산불 {clsName} 탐지 (발생확률 {clsConf}%)"
        else:
            continue

        if cnt % 10 == 0:
            await sendTelegram(cfgList = sysOpt['telegram'], msg = msg)

        cnt += 1

[CHECK] getcwd : /content/drive/MyDrive/COLAB/testSys-FireWatcher/droneSmokeFireDetection

video 1/1 (frame 1/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 1132.8ms
video 1/1 (frame 2/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 971.1ms
video 1/1 (frame 3/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 872.5ms
video 1/1 (frame 4/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 693.3ms
video 1/1 (frame 5/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 834.0ms
video 1/1 (frame 6/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 1 smoke, 602.1ms
video 1/1 (frame 7/1776) /content/drive/MyDrive/COLAB/testSys-FireWatcher/INPUT/20250707_drone_fire.mp4: 480x800 (no detections)